# INTRODUCCIÓN A TEORÍA BLOCKCHAIN

## Sección 1: ¿Qué es una Blockchain?

### ¿Qué es una blockchain?

Una **blockchain** (o "cadena de bloques") es una base de datos distribuida que guarda información de forma segura, transparente e inmutable.

Imagina una hoja de cálculo compartida por muchas personas en todo el mundo. Cada vez que alguien añade una nueva fila (bloque), todos los demás ven esa actualización, y nadie puede modificar filas pasadas sin que todos se enteren.

### Características clave:

- 🔒 **Inmutabilidad**: una vez que algo se guarda, no se puede cambiar sin romper la cadena.
- 👥 **Descentralización**: no hay una autoridad central, sino múltiples nodos.
- 🔗 **Encadenamiento criptográfico**: cada bloque tiene una referencia al anterior, formando una cadena.
- ⛏️ **Prueba de trabajo (Proof of Work)**: para añadir un bloque, se debe resolver un problema difícil (como un Sudoku computacional).

Ahora vamos a construir nuestra propia blockchain en Python desde cero. 🧠👨‍💻


## Sección 2: ¿Qué es un bloque?

### ¿Qué es un bloque?

Un bloque es una unidad de información. Contiene:

- 📦 Datos (por ejemplo, una transacción)
- ⏱️ Marca de tiempo
- 🔗 Hash del bloque anterior (enlace en la cadena)
- 🧩 Nonce (número para resolver la prueba de trabajo)
- 🧠 Su propio hash (identificador único generado con un algoritmo criptográfico)

Vamos a crear la clase `Block` para representar esto.


### Código - clase Block

In [ ]:
import hashlib   # Para funciones hash criptográficas
import time      # Para capturar el momento exacto de creación de un bloque
import json      # Para serializar los datos del bloque de forma consistente

class Block:
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index                      # Posición del bloque en la cadena
        self.timestamp = timestamp              # Fecha y hora de creación
        self.data = data                        # Información que contiene el bloque (por ejemplo, transacciones)
        self.previous_hash = previous_hash      # Hash del bloque anterior en la cadena
        self.nonce = nonce                      # Número aleatorio que se modifica hasta que se cumpla la dificultad
        self.hash = self.calculate_hash()       # Hash del bloque calculado en base a su contenido

    def calculate_hash(self):
        # Creamos un diccionario solo con los atributos relevantes (sin incluir el propio hash)
        block_content = {
            "index": self.index,
            "timestamp": self.timestamp,
            "data": self.data,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }
        # Serializamos el contenido del bloque y lo codificamos en bytes
        block_string = json.dumps(block_content, sort_keys=True).encode()
        # Calculamos el hash SHA-256 del contenido serializado
        return hashlib.sha256(block_string).hexdigest()

    def __repr__(self):
        # Representación legible del bloque para imprimirlo bonito
        return json.dumps(self.__dict__, indent=4)



## Sección 3: La cadena de bloques

### ¿Qué es la cadena?

Una **cadena de bloques** no es más que una lista de bloques conectados entre sí por sus hashes.

El primer bloque se llama **bloque génesis**. Cada nuevo bloque contiene el hash del anterior, lo que garantiza la integridad.

A continuación creamos nuestra clase `Blockchain`.


## Código – clase Blockchain

In [9]:
class Blockchain:
    def __init__(self, difficulty=4):
        self.chain = [self.create_genesis_block()]  # Lista de bloques que comienza con el bloque génesis
        self.difficulty = difficulty                # Número de ceros iniciales que debe tener el hash (dificultad del minado)

    def create_genesis_block(self):
        # Crea el primer bloque de la cadena, llamado "bloque génesis"
        return Block(0, time.time(), "Genesis Block", "0")

    def get_last_block(self):
        # Devuelve el último bloque de la cadena
        return self.chain[-1]

    def add_block(self, new_data):
        # Crea un nuevo bloque con los datos que le pasamos
        last_block = self.get_last_block()                           # Obtenemos el último bloque actual
        new_block = Block(len(self.chain), time.time(), new_data, last_block.hash)
        new_block = self.proof_of_work(new_block)                    # Aplicamos la prueba de trabajo para validar el bloque
        self.chain.append(new_block)                                 # Añadimos el bloque a la cadena

    def proof_of_work(self, block):
        # Intenta encontrar un hash que empiece con un número determinado de ceros ("0" * difficulty)
        print(f"⛏️ Mining block {block.index}...")
        while block.hash[:self.difficulty] != "0" * self.difficulty:
            block.nonce += 1                      # Incrementamos el nonce para cambiar el hash
            block.hash = block.calculate_hash()   # Calculamos un nuevo hash
        print(f"✅ Block mined: {block.hash}")     # Éxito al encontrar un hash válido
        return block

    def is_chain_valid(self):
        # Verifica si la cadena completa es válida
        for i in range(1, len(self.chain)):
            current = self.chain[i]               # Bloque actual
            prev = self.chain[i-1]                # Bloque anterior

            if current.hash != current.calculate_hash():
                # El hash del bloque no es válido (puede haber sido alterado)
                return False

            if current.previous_hash != prev.hash:
                # El hash anterior no coincide con el hash del bloque anterior
                return False

        return True  # Todos los bloques son válidos

    def print_chain(self):
        # Muestra todos los bloques de la cadena
        for block in self.chain:
            print(block)
            print("-" * 40)


## Sección 4: Probamos nuestra blockchain

Vamos a crear nuestra blockchain y añadir algunos bloques con datos simulados de transacciones.


In [10]:
# Creamos una blockchain con dificultad 3 (más fácil de minar para pruebas)
mi_blockchain = Blockchain(difficulty=3)

# Añadimos algunos bloques con datos simulados
mi_blockchain.add_block({"from": "Alice", "to": "Bob", "amount": 100})
mi_blockchain.add_block({"from": "Bob", "to": "Charlie", "amount": 50})
mi_blockchain.add_block({"from": "Charlie", "to": "David", "amount": 25})

# Imprimimos todos los bloques de la cadena
mi_blockchain.print_chain()

# Verificamos si la cadena es válida
print("¿La cadena es válida?", mi_blockchain.is_chain_valid())


⛏️ Mining block 1...
✅ Block mined: 000bbf9290baf45b1157c54afe5c2a889a4ebbdcfe6b47c406b0ec0b7685764c
⛏️ Mining block 2...
✅ Block mined: 0003c531b24bef13637f37fa009817aa77c01a652b718d4b30082b10dca18dfa
⛏️ Mining block 3...
✅ Block mined: 0004b260bcfc4b7e1a1b3855fa8f55abef2f9c533b5e0ec6320994009608ecab
{
    "index": 0,
    "timestamp": 1745943867.05388,
    "data": "Genesis Block",
    "previous_hash": "0",
    "nonce": 0,
    "hash": "22fe0c3f15d515c4f6f97382b3e010c199968a690af3ac9bfc61ad7395e187d5"
}
----------------------------------------
{
    "index": 1,
    "timestamp": 1745943867.0540102,
    "data": {
        "from": "Alice",
        "to": "Bob",
        "amount": 100
    },
    "previous_hash": "22fe0c3f15d515c4f6f97382b3e010c199968a690af3ac9bfc61ad7395e187d5",
    "nonce": 1571,
    "hash": "000bbf9290baf45b1157c54afe5c2a889a4ebbdcfe6b47c406b0ec0b7685764c"
}
----------------------------------------
{
    "index": 2,
    "timestamp": 1745943867.0888588,
    "data": {
        

## Sección 5: ¿Qué aprendimos?

### Conclusión

Hemos construido una blockchain funcional y didáctica:

- Creamos bloques con hash únicos
- Encadenamos esos bloques usando el hash anterior
- Aplicamos prueba de trabajo (Proof of Work)
- Validamos la integridad de la cadena

🚀 Próximos pasos posibles:

- Simular ataques
- Añadir firma digital con claves públicas/privadas
- Crear una API con Flask para interactuar con la blockchain
- Conectarla con otros nodos (red P2P)
